In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from utils.get_sequences import get_sequences
from utils.split_data import split_data
from sklearn.preprocessing import StandardScaler
from data_files.data_module import EnergyChickenDataModule
from model_files.model import LSTM
from model_files.trainer import Trainer
from torch.optim import Adam, SGD, Adadelta, AdamW, Adagrad
import matplotlib.pyplot as plt
import json

In [31]:
RANDOM_SEED = 42
SPLITER_COUNT = 0.8
WINDOW_LENGHT = 24 # window size of one month
BATCH_SIZE = 32
MAX_COUNT_DECREASING = 10
LEARNING_RATE = 0.001
N_EPOCHS = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [32]:
df = pd.read_csv('data_files/processed.csv')
df.drop(columns=df.columns[0], inplace=True)
scaler = StandardScaler()
data = torch.FloatTensor(scaler.fit_transform(df.iloc[:, 0].to_numpy().reshape((-1, 1)))).to(DEVICE)[:1000]

sequences = get_sequences(data, WINDOW_LENGHT)
train, test = split_data(sequences, SPLITER_COUNT)

In [33]:
model = LSTM(device=DEVICE)
optimizers = [Adam, SGD, Adadelta, AdamW, Adagrad] # optimizers that we will test

In [34]:

criterion = nn.MSELoss()

comparison_logs = {}

for optimizer in optimizers:
    data_module = EnergyChickenDataModule(train, test)
    data_module.setup()
    train_dataloader = data_module.train_dataloader()
    test_dataloader = data_module.test_dataloader()

    optimizer_name = str(optimizer).split('.')[-1][:-2]
    optimizer = optimizer(model.parameters(), lr=LEARNING_RATE)
    print(optimizer_name)

    model = LSTM(device=DEVICE)
    trainer = Trainer(model=model, 
                optimizer=optimizer, 
                loss_function=criterion,
                n_epochs=N_EPOCHS,
                device=DEVICE,
                max_count_decreasing=MAX_COUNT_DECREASING)
    print(train_dataloader)

    trainer.upload_data(train_dataloader, test_dataloader)
    trainer.fit()
    
    comparison_logs[optimizer_name] = trainer.logs
    with open('compatison_results.json', 'w') as file:
        json.dump(comparison_logs, file, ensure_ascii=False, indent=4)
    

Adam


/home/argen7um/Desktop/code/energy_prediction/data_files/dataset.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return (torch.Tensor(sequence), torch.tensor(label).float())
/home/argen7um/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 1 train_loss 0.960212 test_loss 0.798444 time 2.584301
epoch 2 train_loss 0.960212 test_loss 0.798444 time 3.004226
epoch 3 train_loss 0.960212 test_loss 0.798444 time 2.907451
epoch 4 train_loss 0.960212 test_loss 0.798444 time 2.97694
epoch 5 train_loss 0.960212 test_loss 0.798444 time 2.878132
epoch 6 train_loss 0.960212 test_loss 0.798444 time 2.831639
epoch 7 train_loss 0.960212 test_loss 0.798444 time 2.684386
epoch 8 train_loss 0.960212 test_loss 0.798444 time 2.79569
epoch 9 train_loss 0.960212 test_loss 0.798444 time 2.430878
epoch 10 train_loss 0.960212 test_loss 0.798444 time 2.59186
SGD
epoch 1 train_loss 0.995324 test_loss 0.920691 time 2.999958


KeyboardInterrupt: 

Adam
epoch 1 train_loss 0.95369 test_loss 0.894374 time 3.842928
epoch 2 train_loss 0.95369 test_loss 0.894374 time 3.861985

In [ ]:

stat = trainer.logs
train_loss = stat['train_loss']
test_loss = stat['test_loss']
min_train_loss = min(train_loss)
min_train_loss_epoch = train_loss.index(min_train_loss)
min_test_loss = min(test_loss)
min_test_loss_epoch = test_loss.index(min_test_loss)
print(f'train_loss {(1-min_train_loss**0.5)*100}\ntest_loss {(1-min_test_loss**0.5)*100} ')

plt.scatter([min_train_loss_epoch], [min_train_loss], label='Min train loss')
plt.scatter([min_test_loss_epoch], [min_test_loss], label='Min test loss')
plt.plot(train_loss, label='Train loss')
plt.plot(test_loss, label='Test loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()




In [8]:
print(str(AdamW).split('.')[-1][:-2])

AdamW
